# Code to visualize and correct TA data 

Code to import and visualize the TA data. You also have the possibility to chrip correct,remove the coherence using the solvent scan as a reference, and smooth the data. Prepare the data to be usable by pyglotaran

Arthur Vard - varthur@mit.edu

# 0. Prelude

In [ ]:
import numpy as np
import tkinter as tk
from tkinter import Tk, filedialog
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.io
import xarray as xr
from tkinter.filedialog import askopenfilename, askopenfilenames
from matplotlib.ticker import MaxNLocator
import os
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
%matplotlib widget

# allow multiple outputs in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

def heatmap_interactive(_x, _y, _data, _title, _cmap='jet', _symlog=False):
    fig = plt.figure(figsize=(8, 8))
    gs = gridspec.GridSpec(2, 2, width_ratios=[1, 0.5], height_ratios=[0.5, 1], hspace=0.2, wspace=0.2)
    ax_main = plt.subplot(gs[1, 0])
    main_plot = ax_main.pcolormesh(_x, _y, _data, cmap=_cmap)
    ax_main.set(xlabel='Delay / ps', ylabel='Wavelength / nm')
    # set mixed log-lin scale with threshold value linthresh
    if _symlog:
        ax_main.set_xscale('symlog', linthresh=0.01)
    # set axis range to min and max values
    ax_main.set_xlim(_x[0],_x[-1])
    ax_main.set_ylim(_y[0],_y[-1]) 

    ax_kin = plt.subplot(gs[0, 0])
    line_kin, = ax_kin.plot(_x,np.zeros(_x.shape))
    kin_zero_line, = ax_kin.plot([_x[0],_x[-1]],[0,0], color="0.6")
    ax_kin.set_xlim(_x[0],_x[-1])
    if _symlog:
        ax_kin.set_xscale('symlog', linthresh=0.01)

    ax_spec = plt.subplot(gs[1, 1])
    line_spec, = ax_spec.plot(np.zeros(_y.shape),_y)
    spec_zero_line, = ax_spec.plot([0,0],[_y[0],_y[-1]], color="0.6")
    ax_spec.set_ylim(_y[0],_y[-1])        
    
    # This lower bounds list is necessary because the blocks in the 2D-plot cover a certain range
    def create_lower_bounds(_value_list):
        result = np.empty_like(_value_list)
        #first lower bound is equal to the lowest value in the nm-list
        result[0] = _value_list[0]
        #example: lower bound for 100 ps is 97.5 ps if the value prior is 95 ps, and 75 ps if the value prior is 50 ps.
        for i in range(1,len(_value_list)):
            result[i] = (_value_list[i]+_value_list[i-1])/2
        return result    
    
    nm_lower_bounds = create_lower_bounds(_y)
    time_lower_bounds = create_lower_bounds(_x)
    
    def nm_to_index(_nm):
        return np.where(_nm > nm_lower_bounds)[0][-1]
    
    def time_to_index(_time):
        return np.where(_time > time_lower_bounds)[0][-1]
    
    def mouse_move(event):
        x = event.xdata
        y = event.ydata
        if x is not None and y is not None:
            if x>=_x[0] and x<=_x[-1] and y>=_y[0] and y<=_y[-1]:
                # update spectra slice and rescale
                new_spec = _data[:,time_to_index(x)]
                line_spec.set_xdata(new_spec)
                spec_bounds = ax_spec.get_ylim()
                spec_range = new_spec[(_y>=spec_bounds[0]) & (_y<=spec_bounds[1])].max()-new_spec[(_y>=spec_bounds[0]) & (_y<=spec_bounds[1])].min()
                ax_spec.set_xlim(new_spec[(_y>=spec_bounds[0]) & (_y<=spec_bounds[1])].min()-0.1*spec_range,new_spec[(_y>=spec_bounds[0]) & (_y<=spec_bounds[1])].max()+0.1*spec_range)            

                # update kinetic slice and rescale
                new_kin = _data[nm_to_index(y),:]
                line_kin.set_ydata(new_kin)
                kin_bounds = ax_kin.get_xlim()  
                kin_range = new_kin[(_x>=kin_bounds[0]) & (_x<=kin_bounds[1])].max()-new_kin[(_x>=kin_bounds[0]) & (_x<=kin_bounds[1])].min()                
                ax_kin.set_ylim(new_kin[(_x>=kin_bounds[0]) & (_x<=kin_bounds[1])].min()-0.1*kin_range,new_kin[(_x>=kin_bounds[0]) & (_x<=kin_bounds[1])].max()+0.1*kin_range)
                
                # redraw figure
                fig.canvas.draw_idle()
             
    fig.canvas.mpl_connect('motion_notify_event', mouse_move) 
    
    # find max absolute value of 2D data in the specified zoom mode of the plot
    def get_maxvalue(_xlim, _ylim, _xvals, _yvals, _data_array):
        y_filter = (_yvals>=_ylim[0]) & (_yvals<=_ylim[1])
        x_filter = (_xvals>=_xlim[0]) & (_xvals<=_xlim[1])
        
        if not np.all(y_filter == False) and not np.all(x_filter == False):
            return np.amax(np.abs(_data_array[y_filter][:,x_filter]))
        else:
            return 0
    
    def on_xlims_change(event_ax):
        ax_kin.set_xlim(event_ax.get_xlim())
        
        new_max = get_maxvalue(event_ax.get_xlim(),event_ax.get_ylim(),_x,_y,_data)
        if new_max > 0:
            main_plot.set_clim(vmin=-new_max, vmax=new_max)

    def on_ylims_change(event_ax):
        ax_spec.set_ylim(event_ax.get_ylim())
        
        new_max = get_maxvalue(event_ax.get_xlim(),event_ax.get_ylim(),_x,_y,_data)
        if new_max > 0:
            main_plot.set_clim(vmin=-new_max, vmax=new_max)        

    ax_main.callbacks.connect('xlim_changed', on_xlims_change)
    ax_main.callbacks.connect('ylim_changed', on_ylims_change)
    plt.show(block=False)

def poly2(x, a, b, c):
    return a * x**2 + b * x + c


def poly4(x, a, b, c, d, e):
    return a * x**4 + b * x**3 + c * x**2 + d * x + e

def lin(x,a,b):
    return a*x+b

def smooth_data_average_window(data, window):
    return np.apply_along_axis(lambda m: np.convolve(m, window, mode='same'), axis=0, arr=data)

# 1. Import and initial look data

## Wavelength Calibaration

In [ ]:
#Wavelength Calibaration
pixels = np.arange(0, 2048)
pixels_calibation = [898, 962, 1255,1495,1675]
wv_calibation = [514, 532, 600,650,700]

calibration,_ = curve_fit(lin,pixels_calibation,wv_calibation, method= "dogbox")
Fit_calibration = lin(pixels,*calibration)

plt.figure()
plt.plot(pixels_calibation,wv_calibation,  'o', label='Data')
plt.plot(pixels,Fit_calibration, '-', label='Fit')
plt.xlabel('Pixels)')
plt.ylabel('Wavelength (nm)')
plt.legend()
plt.grid("On")
plt.show()
print(f"y = {calibration[0]}x + {calibration[1]}")

lambda_values = calibration[0] * pixels + calibration[1]

In [ ]:
#Pump Wavelenght (in nm)
Pump = 400 

#Get time vector data file
time_file = askopenfilename(filetypes=[("Text files", "*.txt")], title="Select Time vector data")
time = np.loadtxt(time_file)
time = time / 1000  # convert from fs to ps

# Get TA scan files
ta_scan_files = askopenfilenames(filetypes=[("Text files", "*.txt")], title="Select TA scan files")

if len(ta_scan_files) > 1:
    Full_Data = np.zeros((2048, len(time), len(ta_scan_files))) #Array that will contain all the 
    for n, file in enumerate(ta_scan_files):
        data = np.loadtxt(file)
        Full_Data[:, :, n] = data  # save data array to 3D array (lambda, time, scan)
    
    for ii in range(len(ta_scan_files)): #Flip the sign if pump scatter is not negative.
        for jj in range(len(time)):
            if Full_Data[Pump, jj, ii] > 0:
                Full_Data[:, jj, ii] *= -1

    scan = np.mean(Full_Data, axis=2)
    scan = scan - np.mean(scan[:,:5],axis=1)[:, np.newaxis] #Baseline correction 
else:
    scan = np.loadtxt(ta_scan_files[0])
# Now 'scan' contains the processed data

#Create an xarray to manipulate the data (much easier)
dataset = xr.Dataset(
    {
        "data": (["time","spectral",], np.transpose(scan))
    },
    coords={
        "time": time,
        "spectral": lambda_values
    }
)

# Print the dataset
print(dataset)

## Heat map visualization of the data

In [ ]:
heatmap_interactive(dataset.time, dataset.spectral, dataset['data'].transpose('spectral','time'),'Averaged scan plot',_symlog=False)

## Look at a specific TA trace or spectrum

### TA trace

In [ ]:
plot_data = dataset.data.sel(spectral=[597], method="nearest").sel(time=slice(None, 10))
plot_data.plot.line(x="time", aspect=2, size=5);

### TA spectrum

In [ ]:
plot_data = dataset.data.sel(time=[0.01,0.02,0.03,0.04,0.045,0.06,0.07,0.08,0.09,0.1,0.14,0.16], method="nearest").sel(spectral=slice(409, 755))
ax = plot_data.plot.line(x="spectral", aspect=2, size=5)
plt.grid(True) 
plt.show()

# 2.Import solvent data

In [ ]:
# Get TA scan files
ta_scan_files = askopenfilenames(filetypes=[("Text files", "*.txt")], title="Select solvent scan file(s)")

if len(ta_scan_files) > 1:
    Full_Data = np.zeros((2048, len(time), len(ta_scan_files))) #Array that will contain all the 
    for n, file in enumerate(ta_scan_files):
        data = np.loadtxt(file)
        Full_Data[:, :, n] = data  # save data array to 3D array (lambda, time, scan)
    
    for ii in range(len(ta_scan_files)): #Flip the sign if pump scatter is not negative.
        for jj in range(len(time)):
            if Full_Data[Pump, jj, ii] > 0:
                Full_Data[:, jj, ii] *= -1

    scan_solvent = np.mean(Full_Data, axis=2)
    scan_solvent = scan_solvent - np.mean(scan_solvent[:,:5],axis=1)[:, np.newaxis] #Baseline correction 
else:
    scan_solvent = np.loadtxt(ta_scan_files[0])
# Now 'scan' contains the processed data

#Create an xarray to manipulate the data (much easier)
dataset_solvent = xr.Dataset(
    {
        "data": (["time","spectral",], np.transpose(scan_solvent))
    },
    coords={
        "time": time,
        "spectral": lambda_values
    }
)

# Print the dataset
print(dataset_solvent)

In [ ]:
heatmap_interactive(dataset_solvent.time, dataset_solvent.spectral, dataset_solvent['data'].transpose('spectral','time'),'Averaged scan plot',_symlog=False)

## Visualization of the Solvent and data kinetic trace to see effect of substraction 

In [ ]:
# Select the data from both datasets
wv = 500 #in nm 

plot_data_solvent = dataset_solvent.data.sel(spectral=[wv], method="nearest").sel(time=slice(None, 10))
plot_data = dataset.data.sel(spectral=[wv], method="nearest").sel(time=slice(None, 10))

# Create a plot
plt.figure(figsize=(10, 5))

# Plot the solvent data
plt.plot(plot_data_solvent.time, plot_data_solvent, label='Solvent Data')

# Plot the other dataset
plt.plot(plot_data.time, plot_data, label='Cu(dpp) Data')

# Add labels and title
plt.xlabel('Time in ps')
plt.ylabel('OD')
plt.title(f'Solvent Data vs. Cu(bcp)2 at {wv} nm')
# Add a legend
plt.legend()
plt.grid("On")
# Show the plot
plt.show()

# 3.Chirp correction from the solvent data using a polynomial Fit 

In [ ]:
# Initialization of the matices 
#Pick the region you want to do the chirp correction from
lower_spectral_bound = 550 #in nm
higher_spectral_bound = 610 #in nm


spectral_region = dataset_solvent.sel(spectral=slice(lower_spectral_bound, higher_spectral_bound)).spectral
time_coherence = np.zeros_like(spectral_region)

for i in range(0,np.size(time_coherence)): #Find where is the maximum coherence in time 
    time_coherence[i] = time[dataset_solvent.data.sel(spectral=spectral_region[i], method="nearest").argmax(dim='time')]

#Fit type, can be changed for lin, poly2, poly4
params_l, _ = curve_fit(lin, spectral_region, time_coherence)
Fit_coherence = lin(spectral_region,*params_l)

#Substraction
time_chirp_corrected = np.zeros((np.shape(spectral_region)[0],np.shape(time)[0]))
for i in range(0,np.size(time_coherence)):
    time_chirp_corrected[i,:] = time - np.array(Fit_coherence[i])


plt.figure()
plt.plot(time_coherence,spectral_region,  'o', label='Data')
plt.plot(Fit_coherence,spectral_region,  '-', label='Fit')
plt.xlabel('Wavelength (in nm)')
plt.ylabel('Position of the coherence max (in ps)')
plt.legend()
plt.grid("On")
plt.show()

## Chirp Correct the solvent data 

In [ ]:
# Make a deep copy of the original dataset
chirp_corrected_solvent = dataset_solvent.copy(deep=True)

for i in range(0,np.size(time_coherence)):
    spectral_val = spectral_region[i]
    time_chirp = time_chirp_corrected[i, :]
    data_slice = chirp_corrected_solvent.data.sel(spectral=spectral_val)
    
    # Perform interpolation
    interpolated_function = interp1d(time_chirp,data_slice,bounds_error=False, fill_value="extrapolate")
    interpolated_data = interpolated_function(time)
    
    # Assign the interpolated data back to the dataset
    chirp_corrected_solvent.data.loc[:, spectral_val] = interpolated_data

#Set NaN values to 0
chirp_corrected_solvent.data.values = np.nan_to_num(chirp_corrected_solvent.data.values)

In [ ]:
heatmap_interactive(chirp_corrected_solvent.time, chirp_corrected_solvent.spectral, chirp_corrected_solvent['data'].transpose('spectral','time'),'Averaged scan plot',_symlog=False)


## Chirp correct the data using the solvent fit 

In [ ]:
chirp_corrected_data = dataset.copy(deep=True)

for i in range(0,np.size(time_coherence)):
    spectral_val = spectral_region[i]
    time_chirp = time_chirp_corrected[i, :]
    data_slice = chirp_corrected_data.data.sel(spectral=spectral_val)
    
    # Perform interpolation
    interpolated_function = interp1d(time_chirp,data_slice,bounds_error=False, fill_value="extrapolate")
    interpolated_data = interpolated_function(time)
    
    # Assign the interpolated data back to the dataset
    chirp_corrected_data.data.loc[:, spectral_val] = interpolated_data


In [ ]:
heatmap_interactive(chirp_corrected_data.time, chirp_corrected_data.spectral, chirp_corrected_data['data'].transpose('spectral','time'),'Averaged scan plot',_symlog=False)


# 4.Remove the coherence from the sample by substracting the solvent  

## Initial look at both chirp corrected traces 

In [ ]:
# Select the data from both datasets
wv = 500 #in nm

plot_data_solvent = chirp_corrected_solvent.data.sel(spectral=[wv], method="nearest").sel(time=slice(None, 30))
plot_data = chirp_corrected_data.data.sel(spectral=[wv], method="nearest").sel(time=slice(None, 30))

# Create a plot
plt.figure(figsize=(10, 5))

# Plot the solvent data
plt.plot(plot_data_solvent.time, plot_data_solvent, label='Solvent Data')

# Plot the other dataset
plt.plot(plot_data.time, plot_data, label='Data')
# Add labels and title
plt.xlabel('Time in ps')
plt.ylabel('OD')
plt.title(f'Chriped corrected Solvent Data vs. data at {wv} nm')
# Add a legend
plt.legend()
plt.grid("On")
# Show the plot
plt.show()

In [ ]:
# Select the spectrum from both datasets
t = 0.03 #in ps

plot_data_solvent = chirp_corrected_solvent.data.sel(time=[t], method="nearest").sel(spectral=slice(None, None))
plot_data = chirp_corrected_data.data.sel(time=[t], method="nearest").sel(spectral=slice(None, None))

# Create a plot
plt.figure(figsize=(10, 5))

# Plot the solvent data
plt.plot(plot_data_solvent.spectral, np.transpose(plot_data_solvent.data), label='Solvent Data')

# Plot the other dataset
plt.plot(plot_data.spectral, np.transpose(plot_data.data), label='Data')
# Add labels and title
plt.xlabel('wl in nm')
plt.ylabel('OD')
plt.title(f'Chriped corrected Solvent Data vs. data at {t} ps')
# Add a legend
plt.legend()
plt.grid("On")
# Show the plot
plt.show()

## Subtraction

In [ ]:
processed_data = chirp_corrected_data - chirp_corrected_solvent
heatmap_interactive(processed_data.time, processed_data.spectral, processed_data['data'].transpose('spectral','time'),'Averaged scan plot',_symlog=False)

In [ ]:
#Visualize effect of the substraction on a kinetic trace

wv = 500
#in nm

plot_data_sub = processed_data.data.sel(spectral=[wv], method="nearest").sel(time=slice(None, 80))
plot_data = chirp_corrected_data.data.sel(spectral=[wv], method="nearest").sel(time=slice(None, 30))


# Create a plot
plt.figure(figsize=(10, 5))

# Plot the other dataset
plt.plot(plot_data.time, plot_data, label='Before substraction Data')
plt.plot(plot_data_sub.time, plot_data_sub, label='After subtraction Data')

# Add labels and title
plt.xlabel('Time in ps')
plt.ylabel('OD')
plt.title(f'Chriped corrected & substracted Data at {wv} nm')
# Add a legend
plt.legend()
plt.grid("On")
# Show the plot
plt.show()

# 5.Smoothing the data

In [ ]:
# Choose which kind of smoothing you want (Moving average or Savitzky–Golay) 

smooth_method = "Savitzky–Golay"

if smooth_method == "Savitzky–Golay": 
    smoothed_data = savgol_filter(processed_data.data.values, window_length = 50, polyorder= 3)

    smoothed_dataset = xr.Dataset(
    {
        "data": (["time", "spectral"], smoothed_data)
    },
    coords={
        "time": processed_data.time.values,
        "spectral": processed_data.spectral.values
    }
    )
if smooth_method == "Moving Average": 
    window_size = 2 #Define size of the average window
    window = np.ones(window_size) / window_size
    smoothed_data = smooth_data_average_window(processed_data.data.values, window) 
    smoothed_dataset = xr.Dataset(
    {
        "data": (["time", "spectral"], smoothed_data)
    },
    coords={
        "time": processed_data.time.values,
        "spectral": processed_data.spectral.values
    }
)


In [ ]:
#Look at a specific kinetic trace
v = 500 #in nm

plot_data = smoothed_dataset.data.sel(spectral=[wv], method="nearest").sel(time=slice(None, 80))

# Create a plot
plt.figure(figsize=(10, 5))

# Plot the other dataset
plt.plot(plot_data.time, plot_data, label='Cu(dsbp) Data')

# Add labels and title
plt.xlabel('Time in ps')
plt.ylabel('OD')
plt.title(f'Fully processed Data at {wv} nm')
# Add a legend
plt.legend()
plt.grid("On")
# Show the plot
plt.show()

In [ ]:
heatmap_interactive(smoothed_dataset.time, smoothed_dataset.spectral, smoothed_dataset['data'].transpose('spectral','time'),'Averaged scan plot',_symlog=False)

# 6.Look at the processed data 

## Spectrum

In [ ]:
time

In [ ]:
plot_data = smoothed_dataset.data.sel(time=[-1,0.01,0.02,0.03,0.04,0.045,0.06,0.07,0.08,0.09,0.1,0.14,0.16], method="nearest").sel(spectral=slice(460, 700))
ax = plot_data.plot.line(x="spectral", aspect=2, size=5);
plt.grid(True) 
plt.xlabel("Wavelength (nm)")
plt.ylabel("OD")
plt.show()

In [ ]:
plot_data = smoothed_dataset.data.sel(time=[0.16,0.18,0.2,0.25,0.3,0.4,0.5,0.7,0.9,1,1.2], method="nearest").sel(spectral=slice(460, 700))
ax = plot_data.plot.line(x="spectral", aspect=2, size=5);
plt.grid(True) 
plt.xlabel("Wavelength (nm)")
plt.ylabel("OD")
plt.show()

In [ ]:
plot_data = smoothed_dataset.data.sel(time=[2,3,5,10,20,40,60,80], method="nearest").sel(spectral=slice(460, 700))
ax = plot_data.plot.line(x="spectral", aspect=2, size=5);
plt.grid(True) 
plt.xlabel("Wavelength (nm)")
plt.ylabel("OD")
plt.show()

## Kinetics

In [ ]:
wv1 = 520 #in nm
wv2 = 560 #in nm

plot_data1 = smoothed_dataset.data.sel(spectral= wv1, method="nearest").sel(time=slice(-2, 1))
plot_data2 = smoothed_dataset.data.sel(spectral= wv2, method="nearest").sel(time=slice(-2, 1))

# Create a plot
plt.figure(figsize=(10, 5))

# Plot the other dataset
plt.plot(plot_data1.time, plot_data1/np.max(plot_data1), label=f'{wv1} nm')
plt.plot(plot_data2.time, plot_data2/np.max(plot_data2), label=f'{wv2} nm')
# Add labels and title
plt.xlabel('Time in ps')
#plt.ylabel('OD')
# Add a legend
plt.legend()
plt.grid("On")
# Show the plot
plt.show()

# 7.Save/import xarray  

In [ ]:
# Save the dataset to a NetCDF file
smoothed_dataset.to_netcdf("Cu(dmp)2_NotChirpedCorrected.nc")

## Import an xarray for vizualization

In [ ]:
dataset_file = askopenfilename(filetypes=[("NetCDF files", "*.nc")], title="Select .nc data file")
smoothed_dataset = xr.load_dataset(dataset_file)

print(smoothed_dataset)

# Extra

## 0.0. Get IRF Parameters

##

#### FWHM of the coherence spike

In [ ]:
def calculate_fwhm(spectral_data, time_coords):
    spectral_data = np.abs(spectral_data)
    peak_value = np.max(spectral_data)
    half_max = peak_value / 2.0
    indices_above_half = np.where(spectral_data >= half_max)[0]
    
    if len(indices_above_half) < 2:
        return np.nan  # Not enough data to calculate FWHM
    
    max_value = np.where(spectral_data == peak_value )[0]
    valid_indices = [max_value[0]] #Create an where where we gonna remove the data that are above the half but that don't correspond to the peak
    current_value = max_value[0]
    for i in  np.where(indices_above_half > max_value)[0]: #Look in the positive increment  
        if indices_above_half[i] == current_value + 1:
            valid_indices.append(indices_above_half[i])
            current_value += 1
        else:
            break
    current_value = max_value[0]
    for i in  reversed(np.where(indices_above_half < max_value)[0]): #Look in the negative increment  
        if indices_above_half[i] == current_value - 1:
            valid_indices.insert(0,indices_above_half[i])
            current_value -= 1
        else:
            break

    # Interpolate to get a more accurate crossing point
    f_left = interp1d(spectral_data[valid_indices[0]-1:valid_indices[0]+1], time_coords[valid_indices[0]-1:valid_indices[0]+1], kind='linear')
    f_right = interp1d(spectral_data[valid_indices[-1]:valid_indices[-1]+2], time_coords[valid_indices[-1]:valid_indices[-1]+2], kind='linear')
    left_half_max_time = f_left(half_max)
    right_half_max_time = f_right(half_max)
    
    # Calculate FWHM as the difference between these two time points
    fwhm = right_half_max_time - left_half_max_time

    return fwhm

In [ ]:
fwhm_values = []
ROI = chirp_corrected_solvent.sel(spectral = slice(500,550))
time_coords = ROI['time'].values
for spectral_value in ROI.spectral.values:
    spectral_data = ROI['data'].sel(spectral=spectral_value).values
    fwhm = calculate_fwhm(spectral_data, time_coords)*1000 #convert to fs
    fwhm_values.append(fwhm)

In [ ]:
plt.figure()
plt.plot(ROI.spectral,fwhm_values,'x')
plt.grid("On")
plt.xlabel("Wavelength (nm)")
plt.ylabel("FWHM (fs)")
plt.show()

In [ ]:
#### Fit with the FWHM trend with a polynome

params_l, _ = curve_fit(poly3, ROI.spectral, fwhm_values, method= "dogbox")
Fit_IRF = poly3(ROI.spectral,*params_l)

plt.figure()
plt.plot(ROI.spectral,fwhm_values,  'o', label='Data')
plt.plot(ROI.spectral,Fit_IRF, '-', label='Fit')
plt.xlabel('Wavelength (in nm)')
plt.ylabel('FWHM (fs)')
plt.legend()
plt.grid("On")
plt.show()
print(params_l)

## Position of the coherence spike

In [ ]:
max_t = []
ROI = np.abs(chirp_corrected_solvent.sel(spectral = slice(500,550)))
time_coords = ROI['time'].values

for spectral_value in ROI.spectral.values:
    spectral_data = ROI['data'].sel(spectral=spectral_value).values
    peak_value = np.max(spectral_data)
    max_value = np.where(spectral_data == peak_value )[0]
    max_t.append(time_coords[max_value][0])

In [ ]:
plt.figure()
plt.plot(ROI.spectral,max_t,'x')
#plt.axhline(y=116, color='r', linestyle='--', linewidth=2)
plt.grid("On")
plt.xlabel("Wavelength (nm)")
plt.ylabel("Max position (ps)")
plt.show()

In [ ]:
params_lin, _ = curve_fit(lin,ROI.spectral.values,max_t, method= "dogbox")
Fit_max = lin(ROI.spectral,*params_lin)

plt.figure()
plt.plot(ROI.spectral,max_t,  'o', label='Data')
plt.plot(ROI.spectral,Fit_max, '-', label='Fit')
plt.xlabel('Wavelength (in nm)')
plt.ylabel('Max position (ps)')
plt.legend()
plt.grid("On")
plt.show()
print(params_lin)

In [ ]:
plot_data = dataset.data.sel(spectral=520, method="nearest").sel(time=slice(None, None))
plot_data.plot.line(x="time", aspect=2, size=5)
plt.grid(True) 
plt.ylabel('OD')
plt.xlabel('Time fs')